In [2]:
import pandas as pd
import logging 

In [ ]:
# Resource
# https://www.kaggle.com/garethjns/microsoft-lightgbm-with-parameter-tuning-0-823

In [ ]:
logging.warning('loading data')
sub = pd.read_csv('/Users/pliu/Downloads/credit')